## 绑定运行时参数

In [1]:
from langchain_openai import ChatOpenAI, OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
model = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0,
)

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "用代数符号写出下面的方程，然后求解。 使用格式\n\n方程:...\n解决方案:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)

In [4]:
chain = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(chain.invoke("x的3次方加7等于34"))

方程：x^3 + 7 = 34

解决方案：将方程两边同时减去7，得到x^3 = 27。然后开立方根，得到x = 3。


## 在运行时配置链的内部结构

通常，您可能想要尝试多种不同的做事方式，甚至向最终用户展示多种不同的做事方式。为了使这种体验尽可能简单，我们定义了两种方法。

首先是 configurable_fields 方法。这允许您配置可运行的特定字段。

其次，一个 configurable_alternatives 方法。使用此方法，您可以列出可以在运行时设置的任何特定可运行对象的替代方案。

In [5]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField

In [6]:
model = model.configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

In [7]:
model.invoke("生成一个随机的整数，不要回答其他任何内容")

AIMessage(content='23')

In [8]:
model.invoke("生成一个随机的整数，不要回答其他任何内容")

AIMessage(content='23')

In [9]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke("生成一个随机的整数，不要回答其他任何内容")

AIMessage(content='5')

## `@chain` decorator

使用“@chain”装饰器创建一个可运行程序,您还可以通过添加 @chain 装饰器将任意函数变成链。这在功能上相当于包装在 RunnableLambda 中。

通过正确跟踪您的链，这将具有提高可观察性的好处。对该函数内的可运行对象的任何调用都将被跟踪为嵌套子函数。

它还允许您像任何其他可运行程序一样使用它，将其组合成链等。

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

In [6]:
prompt1 = ChatPromptTemplate.from_template("给我讲一个关于 {topic} 的故事")
prompt2 = ChatPromptTemplate.from_template("{story}\n\n对上面这个故事进行修改，让故事变得更加口语化和幽默有趣")

In [9]:
from langchain_openai import ChatOpenAI, OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
model = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.8,
)

In [10]:
@chain
def custom_chain(text):
    prompt_val1 = prompt1.invoke({"topic":text })
    output1 = model.invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    
    chain2 = prompt2 | model | StrOutputParser()
    return chain2.invoke({"story": parsed_output1})

In [12]:
@chain
def custom_chain2(text):
    chain1 = prompt1 | model | StrOutputParser()
    parsed_output1 = chain1.invoke({"topic":text })
    
    chain2 = prompt2 | model | StrOutputParser()
    return chain2.invoke({"story": parsed_output1})

In [16]:
from langchain_core.runnables import RunnablePassthrough

@chain
def custom_chain3(text):
    chain = prompt1 | model | StrOutputParser()| {"story": RunnablePassthrough()} | prompt2 | model | StrOutputParser()
    return chain.invoke({"topic":text })

In [17]:
custom_chain.invoke("有志者事竟成")

'老话说得好：“有志者事竟成”，有一个小故事特别能说明这个道理。一个家伙想爬一座高山，这山高得离谱，看着都让人头疼。但是他没被吓到，一心想着一定要登顶。他一试再试，摔了无数次跟头，但是他不气馁，每次跌倒就笑一笑，拍拍屁股继续往上爬。别人劝他放弃，说：“你看看这山这么高，怎么可能爬上去呢？”但他不听，坚持不懈地往上爬。最后，奇迹发生了！他终于登顶了！这个故事告诉我们，只要你有心，不怕失败，坚持到底，不论多困难的事都能成功！'

In [13]:
custom_chain2.invoke("有志者事竟成")

'阿比吉特是印度一个穷乡僻壤的娃儿，家里一贫如洗，连电、自来水、路都木有！但他可不服输，一心想读书改命。每天在河边寻摸纸张和棍子，自学读写。有一天，来了个城里人，看见阿比吉特在石头上写画，惊呆了。当得知他的处境和愿望后，这个人心动了，决定拉他一把。联系一个NGO（非政府组织），给了他奖学金让他去城里上学。\n\n凭着自己的努力和坚持，阿比吉特最终拿到工程学学位并在一家知名科技公司找到工作。但没忘本的他，现在致力于帮助像自己一样的穷孩子受教育。这个故事告诉我们，只要有决心和毅力，就能克服困难，实现梦想。'

In [18]:
custom_chain3.invoke("有志者事竟成")

'东汉初年，有一位名叫耿弇（音：yǎn）的小伙子从小就立志当兵保卫国家。他聪明伶俐，勤奋好学，尤其喜欢琢磨打仗的事儿。长大后，耿弇投奔了当时的名将刘秀，很快就被看重和提拔。\n\n有一天，刘秀派耿弇去收拾山东那边的土霸王张步。张步占着几座城池，兵强马壮，可不是好惹的。刚开始几次打仗，耿弇都输了，但他没灰心丧气，反而更坚定了必胜的决心。他一边训练部队、鼓舞士气，一边找张步的破绽和弱点。\n\n仔细观察了一段时间后，耿弇发现张步虽然实力强，但城里城外联系不紧密。于是，他决定各个击破，先拿下一个个小城池。\n\n首先，耿弇派了点人马假装攻打张步的大本营临淄城，吸引他的主力来救。当敌人出城后，耿弇立即带主力部队轻松夺下了空城的临淄城。接着，他又利用敌人分散的弱点，闪电般地攻占了一座又一座城池。\n\n等张步发现不对劲的时候，已经晚了，他不得不投降，山东也就此平定了。\n\n刘秀知道耿弇成功后非常高兴，直感叹：“有志气的人，啥事都能干成！”从此，这句话流传开来，激励着后来的勇士们勇往直前、永不放弃。'